In [15]:
from qampy import signals, impairments, equalisation, phaserec, helpers
from qampy.theory import ber_vs_es_over_n0_qam as ber_theory
from qampy.helpers import normalise_and_center as normcenter
from qampy.core.filter import rrcos_pulseshaping as lowpassFilter
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from Funcoes import *

In [16]:
plt.rcParams['font.size'] = 18
plt.rcParams['figure.figsize'] = [16, 8]
plt.rcParams['lines.linewidth'] = 2

In [27]:
M = 64        # ordem da modulação
Fb = 40e9      # taxa de símbolos
SpS = 4         # amostras por símbolo
Fs = SpS*Fb    # taxa de amostragem
SNR = 40        # relação sinal ruído (dB)
rolloff = 0.01  # Rolloff do filtro formatador de pulso
sfm = sinal_qam_fase_min(M,Fb,SpS,SNR)
ordem = 64
dataset , X , y = dataset_02(sfm,ordem)

In [28]:
X_train = X[:2000]
X_test = X[2000:4000]

y_train = y[:2000]
y_test = y[2000:4000]

In [29]:
y_test.shape

(2000,)

In [30]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [31]:
num_features = ordem
length = 16
generator = TimeseriesGenerator(X_train, y_train, length=length, batch_size=1)

In [32]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(length, num_features)))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 76,201
Trainable params: 76,201
Non-trainable params: 0
_________________________________________________________________


In [34]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)

In [35]:
validation_generator = TimeseriesGenerator(
    X_test, y_test, length=length, batch_size=1)

In [36]:
model.fit_generator(generator, epochs=20,
                    validation_data=validation_generator,
                    callbacks=[early_stop])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1984 steps, validate for 1984 steps
Epoch 1/20
1984/1984 [==============================] - 137s 69ms/step - loss: 0.0437 - val_loss: 0.0068
Epoch 2/20
1984/1984 [==============================] - 118s 60ms/step - loss: 0.0109 - val_loss: 0.0045
Epoch 3/20
1984/1984 [==============================] - 150s 76ms/step - loss: 0.0088 - val_loss: 0.0061
Epoch 4/20
1984/1984 [==============================] - 140s 71ms/step - loss: 0.0072 - val_loss: 0.0038
Epoch 5/20
1984/1984 [==============================] - 131s 66ms/step - loss: 0.0057 - val_loss: 0.0025
Epoch 6/20
1984/1984 [==============================] - 153s 77ms/step - loss: 0.0050 - val_loss: 0.0029
Epoch 7/20
1984/1984 [==============================] - 141s 71ms/step - loss: 0.0050 - val_loss: 0.0067
Epoch 8/20
1984/1984 [==============================] - 123s 62ms/step - loss: 0.0045 - val_loss: 0.0021
Epoch 9/20
1984/1984 [==============================] - 118s 59m